In [8]:
!pip install scikit-learn pandas joblib

In [9]:
df = pd.read_csv('Housing.csv')
print("Rows & Columns:", df.shape)
df.head()

Rows & Columns: (545, 13)


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [11]:
features = ["area", "bedrooms", "bathrooms", "stories"]
target = "price"
df = df[features + [target]].dropna()
X, y = df[features], df[target]

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score

# Simple pipeline (saves scaler + model together)
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

# Fit the model on the training data
pipe.fit(X_train, y_train)

# Evaluate on the test set
preds = pipe.predict(X_test)
mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)
print({"MAE": mae, "R2": r2})

{'MAE': 1158970.480316688, 'R2': 0.5137585349037066}


In [13]:
import joblib

artifact = {
    "model": pipe,             # includes scaler + linear regression
    "features": features,      # input order for inference
    "target": target,
    "metrics": {"mae": float(mae), "r2": float(r2)}
}
joblib.dump(artifact, "model.pkl")
print("Saved model.pkl")


Saved model.pkl


In [17]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
import numpy as np
import joblib

app = FastAPI(
    title="House Price Prediction API",
    description="Predict house prices using a saved LinearRegression pipeline",
    version="1.0.0",
)

# Globals populated on startup
artifact = None
model = None
features_order = None
metrics = None

class PredictInput(BaseModel):
    area: float = Field(gt=0, description="Square feet")
    bedrooms: int = Field(ge=0, description="Number of bedrooms")
    bathrooms: int = Field(ge=0, description="Number of bathrooms")
    stories: int = Field(ge=1, description="Number of stories")

class PredictOutput(BaseModel):
    prediction: float

@app.on_event("startup")
def load_model():
    global artifact, model, features_order, metrics
    try:
        artifact = joblib.load("model.pkl")
        model = artifact["model"]
        features_order = artifact["features"]
        metrics = artifact.get("metrics", {})
    except Exception as e:
        # Fail fast if model can't be loaded
        raise RuntimeError(f"Could not load model.pkl: {e}")

@app.get("/")
def health_check():
    return {"status": "ok", "message": "API is running"}

@app.post("/predict", response_model=PredictOutput)
def predict(payload: PredictInput):
    try:
        # Ensure order: [area, bedrooms, bathrooms, stories]
        row = np.array([[payload.area, payload.bedrooms, payload.bathrooms, payload.stories]])
        pred = float(model.predict(row)[0])
        return {"prediction": pred}
    except Exception as e:
        # Proper error handling with clear message
        raise HTTPException(status_code=400, detail=f"Prediction failed: {e}")

@app.get("/model-info")
def model_info():
    try:
        model_name = (
            type(model.named_steps["model"]).__name__
            if hasattr(model, "named_steps") else type(model).__name__
        )
        return {
            "model": model_name,
            "problem_type": "regression",
            "features": features_order,
            "metrics": metrics,
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Info unavailable: {e}")


/tmp/ipython-input-2984078541.py:27: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")
